# ETL: Populate Database from CSV Files

This notebook populates the local SQLite database with data from:
- `neighbourhoods.csv` → `neighbourhood` table
- `listings_cleaned.csv` → `listing` table (with neighbourhood_id foreign key)

**Prerequisites:**
1. Run `sql/setup_local_db_sqlite.sh` to create the database and schema
2. Or manually run: `sqlite3 data/airbnb.db < sql/schema/01_logical_schema_sqlite.sql`

The database file will be created at `data/airbnb.db`


In [ ]:
import pandas as pd
import sqlite3
import os
from pathlib import Path

# Set up paths - handle running from sql/etl/ directory
current_dir = Path().resolve()
# If we're in sql/etl/, go up two levels to project root
if current_dir.name == "etl" and current_dir.parent.name == "sql":
    project_root = current_dir.parent.parent
elif (current_dir / "data").exists():
    project_root = current_dir
else:
    project_root = current_dir.parent.parent

# Database file path
db_path = project_root / "data" / "airbnb.db"

# City configuration - process all cities
cities_config = [
    {"folder": "NYC", "name": "NYC"},
    {"folder": "Boston", "name": "Boston"},
    {"folder": "Washington_DC", "name": "Washington DC"}  # Map folder name to display name
]

data_dir = project_root / "data"
processed_dir = data_dir / "processed"

print(f"Project root: {project_root}")
print(f"\nDatabase file: {db_path} (exists: {db_path.exists()})")
print(f"\nProcessing cities:")
for city_config in cities_config:
    city_folder = city_config["folder"]
    city_name = city_config["name"]
    listings_file = processed_dir / f"{city_folder}_listings_cleaned.csv"
    neighbourhoods_file = data_dir / city_folder / "neighbourhoods.csv"
    print(f"  {city_name}:")
    print(f"    Listings: {listings_file} (exists: {listings_file.exists()})")
    print(f"    Neighbourhoods: {neighbourhoods_file} (exists: {neighbourhoods_file.exists()})")


Project root: /Users/anishj29/Desktop/Github Projects/Airbnb-Price-Predictor

Database file: /Users/anishj29/Desktop/Github Projects/Airbnb-Price-Predictor/data/airbnb.db (exists: True)

Processing cities:
  NYC:
    Listings: /Users/anishj29/Desktop/Github Projects/Airbnb-Price-Predictor/data/processed/NYC_listings_cleaned.csv (exists: False)
    Neighbourhoods: /Users/anishj29/Desktop/Github Projects/Airbnb-Price-Predictor/data/NYC/neighbourhoods.csv (exists: True)
  Boston:
    Listings: /Users/anishj29/Desktop/Github Projects/Airbnb-Price-Predictor/data/processed/Boston_listings_cleaned.csv (exists: False)
    Neighbourhoods: /Users/anishj29/Desktop/Github Projects/Airbnb-Price-Predictor/data/Boston/neighbourhoods.csv (exists: True)
  Washington DC:
    Listings: /Users/anishj29/Desktop/Github Projects/Airbnb-Price-Predictor/data/processed/Washington_DC_listings_cleaned.csv (exists: False)
    Neighbourhoods: /Users/anishj29/Desktop/Github Projects/Airbnb-Price-Predictor/data/Washi

## Database Connection

Connect to the SQLite database file. The database file should be at `data/airbnb.db`.


In [2]:
# SQLite database connection
# The database file path is set above
print("SQLite Database Configuration:")
print(f"  Database file: {db_path}")
print(f"  File exists: {db_path.exists()}")

if not db_path.exists():
    print("\n⚠ Warning: Database file does not exist!")
    print("Please run 'sql/setup_local_db_sqlite.sh' to create the database and schema.")
    print("Or manually run: sqlite3 data/airbnb.db < sql/schema/01_logical_schema_sqlite.sql")
else:
    print("\n✓ Database file found")


SQLite Database Configuration:
  Database file: /Users/anishj29/Desktop/Github Projects/Airbnb-Price-Predictor/data/airbnb.db
  File exists: True

✓ Database file found


In [3]:
# Test database connection
try:
    conn = sqlite3.connect(str(db_path))
    # Enable foreign key constraints
    conn.execute("PRAGMA foreign_keys = ON")
    cur = conn.cursor()
    
    # Test query - check SQLite version
    cur.execute("SELECT sqlite_version();")
    version = cur.fetchone()
    print(f"✓ Successfully connected to database!")
    print(f"SQLite version: {version[0]}")
    
    # Check if tables exist
    cur.execute("""
        SELECT name FROM sqlite_master 
        WHERE type='table' AND name IN ('neighbourhood', 'listing');
    """)
    tables = [row[0] for row in cur.fetchall()]
    
    expected_tables = ['neighbourhood', 'listing']
    missing_tables = [t for t in expected_tables if t not in tables]
    
    if not missing_tables:
        print(f"✓ All tables exist: {', '.join(tables)}")
    else:
        print(f"⚠ Missing tables: {', '.join(missing_tables)}")
        print("Please run the schema creation script first.")
    
    cur.close()
    conn.close()
    
except sqlite3.Error as e:
    print(f"✗ Error connecting to database: {e}")
    raise


✓ Successfully connected to database!
SQLite version: 3.50.4
✓ All tables exist: neighbourhood, listing


## Load and Prepare CSV Data


In [4]:
# Load all city data
all_city_data = {}

for city_config in cities_config:
    city_folder = city_config["folder"]
    city_name = city_config["name"]
    
    listings_file = processed_dir / f"{city_folder}_listings_cleaned.csv"
    neighbourhoods_file = data_dir / city_folder / "neighbourhoods.csv"
    
    if not listings_file.exists():
        print(f"⚠ Skipping {city_name}: {listings_file} not found")
        continue
    
    if not neighbourhoods_file.exists():
        print(f"⚠ Skipping {city_name}: {neighbourhoods_file} not found")
        continue
    
    print(f"\n{'='*60}")
    print(f"Loading data for {city_name}...")
    print(f"{'='*60}")
    
    # Load neighbourhoods
    print(f"Loading neighbourhoods from {neighbourhoods_file}...")
    df_neighbourhoods = pd.read_csv(neighbourhoods_file)
    print(f"  Loaded {len(df_neighbourhoods)} neighbourhoods")
    
    # Load listings
    print(f"Loading listings from {listings_file}...")
    df_listings = pd.read_csv(listings_file)
    print(f"  Loaded {len(df_listings)} listings")
    
    all_city_data[city_name] = {
        "neighbourhoods": df_neighbourhoods,
        "listings": df_listings,
        "folder": city_folder
    }

print(f"\n{'='*60}")
print(f"Summary: Loaded data for {len(all_city_data)} cities")
print(f"{'='*60}")


⚠ Skipping NYC: /Users/anishj29/Desktop/Github Projects/Airbnb-Price-Predictor/data/processed/NYC_listings_cleaned.csv not found
⚠ Skipping Boston: /Users/anishj29/Desktop/Github Projects/Airbnb-Price-Predictor/data/processed/Boston_listings_cleaned.csv not found
⚠ Skipping Washington DC: /Users/anishj29/Desktop/Github Projects/Airbnb-Price-Predictor/data/processed/Washington_DC_listings_cleaned.csv not found

Summary: Loaded data for 0 cities


In [5]:
# Check data quality for each city
for city_name, city_data in all_city_data.items():
    print(f"\n{'='*60}")
    print(f"Data quality check for {city_name}")
    print(f"{'='*60}")
    
    df_neighbourhoods = city_data["neighbourhoods"]
    df_listings = city_data["listings"]
    
    print("\nNeighbourhoods data info:")
    print(df_neighbourhoods.info())
    print("\nNeighbourhoods missing values:")
    print(df_neighbourhoods.isnull().sum())
    
    print("\nListings data info:")
    print(df_listings.info())
    print("\nListings missing values:")
    print(df_listings.isnull().sum())


## Populate Neighbourhood Table

First, we populate the neighbourhood table since the listing table has a foreign key reference to it.


In [6]:
# Prepare and insert neighbourhood data for all cities
print("="*60)
print("STEP: Populating Neighbourhood Table")
print("="*60)

# Connect to database (always create a fresh connection for this operation)
conn = sqlite3.connect(str(db_path))
conn.execute("PRAGMA foreign_keys = ON")
cur = conn.cursor()

# Get existing neighbourhoods to avoid duplicates
cur.execute("SELECT borough, neighbourhood_name FROM neighbourhood;")
existing_neighbourhoods = set((row[0], row[1]) for row in cur.fetchall())

total_new_neighbourhoods = 0

for city_name, city_data in all_city_data.items():
    df_neighbourhoods = city_data["neighbourhoods"]
    print(f"\nProcessing neighbourhoods for {city_name}...")
    print(f"  Preparing {len(df_neighbourhoods)} neighbourhoods...")
    
    neighbourhood_data = []
    for _, row in df_neighbourhoods.iterrows():
        neighbourhood_row = (
            str(row['neighbourhood_group']) if pd.notna(row.get('neighbourhood_group')) else None,  # borough
            str(row['neighbourhood']) if pd.notna(row.get('neighbourhood')) else None,  # neighbourhood_name
        )
        neighbourhood_data.append(neighbourhood_row)
    
    # Filter out duplicates
    new_neighbourhood_data = []
    for row in neighbourhood_data:
        if (row[0], row[1]) not in existing_neighbourhoods:
            new_neighbourhood_data.append(row)
            existing_neighbourhoods.add((row[0], row[1]))
    
    if new_neighbourhood_data:
        insert_neighbourhood_query = """
            INSERT INTO neighbourhood (borough, neighbourhood_name)
            VALUES (?, ?);
        """
        cur.executemany(insert_neighbourhood_query, new_neighbourhood_data)
        conn.commit()
        print(f"  ✓ Inserted {len(new_neighbourhood_data)} new neighbourhoods for {city_name}")
        total_new_neighbourhoods += len(new_neighbourhood_data)
    else:
        print(f"  ✓ All neighbourhoods for {city_name} already exist in database")

cur.execute("SELECT COUNT(*) FROM neighbourhood;")
neighbourhood_count = cur.fetchone()[0]
print(f"\n✓ Total neighbourhoods in database: {neighbourhood_count}")
print(f"  (Added {total_new_neighbourhoods} new neighbourhoods in this run)")

# Keep connection open for subsequent cells (don't close here)


STEP: Populating Neighbourhood Table

✓ Total neighbourhoods in database: 230
  (Added 0 new neighbourhoods in this run)


## Populate Listing Table


In [7]:
# Connect to database
conn = sqlite3.connect(str(db_path))
# Enable foreign key constraints
conn.execute("PRAGMA foreign_keys = ON")
cur = conn.cursor()

print("Connected to SQLite database")


Connected to SQLite database


### Prepare and Insert Listing Data


In [8]:
# Prepare listing data for all cities
print("="*60)
print("STEP: Populating Listing Table")
print("="*60)

# Connect to database (use existing connection if available, otherwise create new)
try:
    # Try to use existing connection
    if conn is None or cur is None:
        raise AttributeError
    # Test if connection is still valid
    cur.execute("SELECT 1")
except (NameError, AttributeError):
    # Create new connection if it doesn't exist or is invalid
    conn = sqlite3.connect(str(db_path))
    conn.execute("PRAGMA foreign_keys = ON")
    cur = conn.cursor()

# Process listings for each city
all_listing_data = []  # Collect all listings from all cities

# First, create lookup dictionaries for neighbourhood_id (once, outside city loop)
cur.execute("SELECT neighbourhood_id, borough, neighbourhood_name FROM neighbourhood;")
neighbourhood_lookup = {}  # For cities with boroughs: (borough, name) -> id
neighbourhood_lookup_by_name = {}  # For cities without boroughs: name -> id
for row in cur.fetchall():
    neighbourhood_id, borough, neighbourhood_name = row
    if borough is None or pd.isna(borough):
        # Match by neighbourhood name only
        neighbourhood_lookup_by_name[neighbourhood_name] = neighbourhood_id
    else:
        # Match by (borough, neighbourhood_name)
        key = (str(borough), str(neighbourhood_name))
        neighbourhood_lookup[key] = neighbourhood_id

# Helper function to convert 't'/'f' to boolean (0/1 for SQLite)
def str_to_bool(value):
    if pd.isna(value):
        return None
    return 1 if str(value).lower() in ['t', 'true', '1', 'yes'] else 0

# Helper function to convert date string to date
def str_to_date(value):
    if pd.isna(value):
        return None
    try:
        return pd.to_datetime(value).date()
    except:
        return None

# Define required columns that must not be null
required_columns = [
    'id', 'host_id', 'host_name', 'host_since', 'host_is_superhost',
    'room_type', 'property_type', 'accommodates', 'bedrooms', 'beds',
    'bathrooms', 'bathrooms_text', 'latitude', 'longitude', 'price',
    'number_of_reviews', 'availability_365',
    'instant_bookable',
    'calculated_host_listings_count', 'estimated_revenue_l365d',
    'neighbourhood_cleansed'  # neighbourhood_group_cleansed is optional (NULL for Boston/DC)
]

for city_name, city_data in all_city_data.items():
    df_listings = city_data["listings"]
    print(f"\nProcessing listings for {city_name}...")
    print(f"  Preparing {len(df_listings)} listings for insertion...")
    
    initial_count = len(df_listings)
    print(f"Initial listings: {initial_count}")

    for _, row in df_listings.iterrows():
        # Check if any required column has null/missing values
        has_nulls = False
        for col in required_columns:
            if col not in row.index or pd.isna(row.get(col)):
                has_nulls = True
                break
        
        if has_nulls:
            continue  # Skip listings with null values
        
        # Look up neighbourhood_id - handle NULL boroughs
        borough = row.get('neighbourhood_group_cleansed')
        neighbourhood_name = row.get('neighbourhood_cleansed')
        neighbourhood_id = None
        
        if pd.notna(neighbourhood_name):
            if pd.notna(borough):
                # Try matching by (borough, neighbourhood_name)
                key = (str(borough), str(neighbourhood_name))
                neighbourhood_id = neighbourhood_lookup.get(key)
            
            # If not found and borough is NULL, try matching by neighbourhood name only
            if neighbourhood_id is None:
                neighbourhood_id = neighbourhood_lookup_by_name.get(str(neighbourhood_name))
        
        # Skip if neighbourhood_id lookup failed
        if neighbourhood_id is None:
            continue
        
        listing_row = (
            int(row['id']),  # listing_id (from 'id' column)
            neighbourhood_id,  # neighbourhood_id (looked up)
            city_name,  # city (NYC, Boston, or Washington DC) - from loop
            int(row['host_id']),  # host_id
            str(row['host_name']),  # host_name
            str_to_date(row.get('host_since')),  # host_since
            str_to_bool(row.get('host_is_superhost')),  # host_is_superhost
            str(row['room_type']),  # room_type
            str(row['property_type']),  # property_type
            int(row['accommodates']),  # accommodates
            int(row['bedrooms']),  # bedrooms
            int(row['beds']),  # beds
            float(row['bathrooms']),  # bathrooms
            str(row['bathrooms_text']),  # bathrooms_text
            float(row['latitude']),  # latitude
            float(row['longitude']),  # longitude
            float(row['price']),  # price
            int(row['number_of_reviews']),  # number_of_reviews
            int(row['availability_365']),  # availability_365
            str_to_date(row.get('first_review')),  # first_review
            str_to_date(row.get('last_review')),  # last_review
            float(row['review_scores_rating']),  # review_scores_rating
            str_to_bool(row.get('instant_bookable')),  # instant_bookable
            int(row['calculated_host_listings_count']),  # calculated_host_listings_count
            float(row['reviews_per_month']),  # reviews_per_month
            float(row['estimated_revenue_l365d']),  # estimated_revenue
        )
        all_listing_data.append(listing_row)

    city_listings = [r for r in all_listing_data if r[2] == city_name]
    city_listings_count = len(city_listings)
    filtered_count = initial_count - city_listings_count
    print(f"✓ Prepared {city_listings_count} listings for {city_name}")
    print(f"  Filtered out {filtered_count} listings with null/missing values")
    city_listings_count = len([r for r in all_listing_data if r[2] == city_name])
    print(f"✓ Prepared {city_listings_count} listings for {city_name}")
    print(f"  Filtered out {filtered_count} listings with null/missing values")

# Summary across all cities
listing_data = all_listing_data  # Use collected data
print(f"\n============================================================")
print(f"Total listings prepared across all cities: {len(listing_data)}")
print(f"============================================================")


STEP: Populating Listing Table

Total listings prepared across all cities: 0


In [9]:
# Check existing listings count
cur.execute("SELECT COUNT(*) FROM listing;")
existing_listings = cur.fetchone()[0]
print(f"Existing listings in database: {existing_listings}")

# Insert listings
insert_listing_query = """
    INSERT INTO listing (
        listing_id, neighbourhood_id, city, host_id, host_name, host_since, host_is_superhost,
        room_type, property_type, accommodates, bedrooms, beds, bathrooms, bathrooms_text,
        latitude, longitude, price, number_of_reviews, availability_365,
        first_review, last_review, review_scores_rating, instant_bookable,
        calculated_host_listings_count, reviews_per_month, estimated_revenue
    )
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ON CONFLICT (listing_id) DO UPDATE SET
        host_is_superhost = EXCLUDED.host_is_superhost,
        accommodates = EXCLUDED.accommodates,
        bedrooms = EXCLUDED.bedrooms,
        beds = EXCLUDED.beds,
        price = EXCLUDED.price,
        number_of_reviews = EXCLUDED.number_of_reviews,
        availability_365 = EXCLUDED.availability_365,
        review_scores_rating = EXCLUDED.review_scores_rating,
        estimated_revenue = EXCLUDED.estimated_revenue;
"""

# Insert in batches for better performance
batch_size = 1000
total_inserted = 0

for i in range(0, len(listing_data), batch_size):
    batch = listing_data[i:i+batch_size]
    cur.executemany(insert_listing_query, batch)
    total_inserted += len(batch)
    if (i // batch_size + 1) % 10 == 0:
        print(f"  Processed {total_inserted} listings...")

conn.commit()

cur.execute("SELECT COUNT(*) FROM listing;")
new_listings = cur.fetchone()[0]
print(f"\n✓ Inserted/updated {new_listings - existing_listings} listings")
print(f"Total listings in database: {new_listings}")


Existing listings in database: 0

✓ Inserted/updated 0 listings
Total listings in database: 0


## Verification

Let's verify the data was inserted correctly.


In [10]:
# Check listing table
cur.execute("SELECT COUNT(*) FROM listing;")
listing_count = cur.fetchone()[0]
print(f"Total listings in database: {listing_count}")

# Sample data
print("\nSample listings:")
cur.execute("""
    SELECT listing_id, price, accommodates, bedrooms, beds, 
           host_is_superhost, number_of_reviews, review_scores_rating, availability_365
    FROM listing 
    LIMIT 5;
""")
for row in cur.fetchall():
    print(f"  Listing ID: {row[0]}, Price: ${row[1]:.2f}, "
          f"Accommodates: {row[2]}, Bedrooms: {row[3]}, Beds: {row[4]}, "
          f"Superhost: {row[5]}, Reviews: {row[6]}, Rating: {row[7]}, Availability: {row[8]}")

# Check data quality
print("\nData quality checks:")
cur.execute("SELECT COUNT(*) FROM listing WHERE price IS NULL;")
null_price = cur.fetchone()[0]
print(f"  Listings with null price: {null_price}")

cur.execute("SELECT COUNT(*) FROM listing WHERE listing_id IS NOT NULL;")
valid_listings = cur.fetchone()[0]
print(f"  Valid listings: {valid_listings}")

cur.execute("SELECT COUNT(*) FROM listing WHERE host_is_superhost IS NOT NULL;")
has_superhost = cur.fetchone()[0]
print(f"  Listings with superhost info: {has_superhost}")

cur.execute("SELECT COUNT(*) FROM listing WHERE review_scores_rating IS NOT NULL;")
has_rating = cur.fetchone()[0]
print(f"  Listings with ratings: {has_rating}")


Total listings in database: 0

Sample listings:

Data quality checks:
  Listings with null price: 0
  Valid listings: 0
  Listings with superhost info: 0
  Listings with ratings: 0


In [11]:
# Close database connection
cur.close()
conn.close()
print("✓ Database connection closed")


✓ Database connection closed
